In [20]:
import os
import pandas as pd
import numpy as np
from skimage import io, color
from skimage.feature import graycomatrix, graycoprops
import glob

In [21]:
# Fonction pour calculer les caractéristiques GLCM
def calculate_glcm_properties(image):
    # Convertir l'image en niveaux de gris
    if len(image.shape) == 3: 
        if image.shape[2] == 4:  
            image = image[:, :, :3] 
        gray_image = color.rgb2gray(image)  
    elif len(image.shape) == 2: 
        gray_image = image
    else:
        raise ValueError("Unexpected format for image.")

    gray_image = (gray_image * 255).astype(np.uint8)

    # Calculer le GLCM et ses propriétés
    glcm = graycomatrix(
        gray_image, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], levels=256
    )
    glcm = glcm + (glcm == 0)

    # Extraire les propriétés du GLCM
    glcm_props = {
        'contrast': graycoprops(glcm, 'contrast').mean(),
        'dissimilarity': graycoprops(glcm, 'dissimilarity').mean(),
        'homogeneity': graycoprops(glcm, 'homogeneity').mean(),
        'ASM': graycoprops(glcm, 'ASM').mean(),
        'energy': graycoprops(glcm, 'energy').mean(),
        'correlation': graycoprops(glcm, 'correlation').mean(),
    }

    return glcm_props

In [22]:
# Fonction pour traiter les images et calculer les caractéristiques GLCM
def process_images(root_folder):
    glcm_results = []
    # Itérer sur chaque sous-dossier dans le dossier racine (représentant les catégories/labels)
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        
        if os.path.isdir(folder_path):
            # Itérer sur tous les fichiers image dans le sous-dossier
            for image_path in glob.glob(os.path.join(folder_path, "*.jpg")):
                try:
                    image = io.imread(image_path)
                    glcm_props = calculate_glcm_properties(image)
                    glcm_props['Label'] = folder_name
                    glcm_props['Image'] = os.path.basename(image_path)
                    glcm_results.append(glcm_props)
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
                    continue
    return glcm_results

In [28]:
# Spécifier le dossier racine où se trouvent les catégories/sous-dossiers
root_folder = "brain-tumor-test"

# Traiter les images et extraire les caractéristiques
glcm_features = process_images(root_folder)

# Convertir les résultats en un DataFrame
df_glcm = pd.DataFrame(glcm_features)

# Sauvegarder le DataFrame dans un fichier Excel
df_glcm.to_excel(root_folder+".xlsx", index=False)

print("Traitement terminé")

Processing complete. Results saved to 'image_features_with_labels_bin.xlsx'.
